In [1]:
import json
import logging
import os
import pandas as pd
import random
import sys
from datetime import date
from dotenv import load_dotenv
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils.utils import GPT

In [2]:
# params
# st_mdl = "gpt-3.5-turbo-1106" # model string
st_mdl = "gpt-4-1106-preview"
st_task = "hotpotqa" # name of task

# logging
## today's date
st_today = date.today().strftime("%Y_%m_%d")

## file name:
st_fn = f"{st_today}_{st_task}_n100_{st_mdl}"

## Configure the logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## Create a file handler and set the formatter
file_handler = logging.FileHandler(f"results/{st_fn}.log")
file_handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)

## Create a logger instance and add the file handler
logger = logging.getLogger(__name__)
logger.addHandler(file_handler)

In [3]:
# load api key from .env and create GPT object
load_dotenv()  # Load environment variables from .env file
openai_api_key = os.getenv("OPENAI_API_KEY")
gpt = GPT(openai_api_key, logger=logger, model=st_mdl)

# Load hotpotqa_test_fullwiki.json into a dictionary
with open('data/hotpotqa_dev_distractor.json', 'r') as f:
    data = json.load(f) # keys in each row: ['_id', 'answer', 'question', 'supporting_facts', 'context', 'type', 'level']

In [4]:
def compare_strings(str1, str2):
    """
    This function compares two strings after converting them to lowercase and stripping leading and trailing spaces.
    """
    return str1.lower().strip() == str2.lower().strip()

def check_bool(str1, bool2):
    """
        Checks if string str1 equal to bool bool2
    """
    last_idx_true = str1.rfind("TRUE")
    last_idx_false = str1.rfind("FALSE")

    # Check if "TRUE" is found and it is the last occurrence
    return last_idx_true != -1 and (last_idx_true > last_idx_false if last_idx_false != -1 else True)
        

In [6]:
# dataset-specific adjustments (we eventually want a list of dicts of format [{"id": "xx", "question": "xxx", "context": "xxx"}, ...]
df = pd.json_normalize(data)
df.head()

,_id,answer,question,supporting_facts,context,type,level
0,5a8b57f25542995d1e6f1371,yes,Were Scott Derrickson and Ed Wood of the same ...,"[[Scott Derrickson, 0], [Ed Wood, 0]]","[[Ed Wood (film), [Ed Wood is a 1994 American ...",comparison,hard
1,5a8c7595554299585d9e36b6,Chief of Protocol,What government position was held by the woman...,"[[Kiss and Tell (1945 film), 0], [Shirley Temp...","[[Meet Corliss Archer, [Meet Corliss Archer, a...",bridge,hard
2,5a85ea095542994775f606a8,Animorphs,"What science fantasy young adult series, told ...","[[The Hork-Bajir Chronicles, 0], [The Hork-Baj...","[[Andre Norton Award, [The Andre Norton Award ...",bridge,hard
3,5adbf0a255429947ff17385a,no,Are the Laleli Mosque and Esma Sultan Mansion ...,"[[Laleli Mosque, 0], [Esma Sultan Mansion, 0]]","[[Esma Sultan (daughter of Abdülaziz), [Esma S...",comparison,hard
4,5a8e3ea95542995a26add48d,"Greenwich Village, New York City","The director of the romantic comedy ""Big Stone...","[[Big Stone Gap (film), 0], [Adriana Trigiani,...","[[Just Another Romantic Wrestling Comedy, [Jus...",bridge,hard


## Generator

In [6]:
predictions = [] # list of predictions, reset after each set of runs
answers = []
indices = [] # question numbers
evals = [] # whether prediction matches answer
questions = [] # questions asked


# Evaluate
num_questions = 100 # number of questions to run on
indices = random.sample(range(len(data)), num_questions)
for idx in indices:
    item = data[idx]
    # if item['level'] == 'hard':
    gold, question, context = item['answer'], item['question'], item['context']
    question_with_context = ""
    for i in range(len(context)):
        context_str = ''.join(context[i][1])
        question_with_context += f"Paragraph {i+1}: {context[i][0]} {context_str} "
    question_with_context += f"Q: {question}"
    logger.info(question_with_context)
    msg = [{"role": "system", "content": "Answer in a single noun, verb, or adjective"}, {"role": "user", "content": question_with_context}]
    predictions.append(gpt.gen(msgs=msg).content)
    answers.append(gold)
    evals.append(compare_strings(predictions[-1], gold))
    questions.append(question_with_context)
    print(predictions[-1])

# Print the predictions and answers
logger.info(predictions)
logger.info(answers)
percent_correct = (sum(evals) / len(evals)) * 100

print(f"Percentage of True values: {percent_correct}%")

# create a dataframe with the data
df = pd.DataFrame({'Index': indices, 'Question': questions, 'Prediction': predictions, 'Answer': answers, 'Eval': evals})

# save the dataframe to csv
df.to_csv(f'results/{st_fn}.csv', index=False)

2024-02-13 22:30:12,035 - INFO - Paragraph 1: Girls! Girls! Girls! Girls! Girls! Girls! is a 1962 Golden Globe-nominated American musical comedy film starring Elvis Presley as a penniless Hawaiian fisherman who loves his life on the sea and dreams of owning his own boat. "Return to Sender", which reached #2 on the "Billboard" pop singles chart, is featured in the movie. The movie opened at #6 on the "Variety" box office chart and finished the year at #31 on the year-end list of the top-grossing movies of 1962. The movie earned $2.6 million at the box office. Paragraph 2: Doctor Who: Series 4 (soundtrack) Doctor Who: Series 4 is a soundtrack album released on 17 November 2008, containing incidental music that was used throughout the fourth series of the BBC science fiction television programme "Doctor Who". The music was composed by the British musical director Murray Gold and was orchestrated by his collaborator Ben Foster who had previously worked with him on the films "Alien Autopsy"

#3


2024-02-13 22:30:13,805 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-13 22:30:13,806 - INFO - Paragraph 1: Inez Dickens Inez E. Dickens is the Assembly member for the 70th District of the New York State Assembly. She is a Democrat. The district includes portions of El Barrio, Hamilton Heights, Harlem, Morningside Heights, the Upper West Side and Washington Heights in Manhattan. She formerly served on the New York City Council from 2006 to 2016. Paragraph 2: Pete Grannis Alexander B. "Pete" Grannis (born 1942/1943) is a former Commissioner of the New York State Department of Environmental Conservation (DEC). Before his tenure as Commissioner, he was a member of the New York State Assembly and represented District 65 as a member of the Democratic Party for the neighborhoods of the Upper East Side of Manhattan and Roosevelt Island. His firing by Governor David Paterson in October 2010 was controversial to many especially environmentalist

Jaguar


2024-02-13 22:30:14,455 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-13 22:30:14,457 - INFO - Paragraph 1: Baby Girl (Sugarland song) "Baby Girl" is the debut single co-written and recorded by American country music group Sugarland. Originally released in 2003 as part of "Premium Quality Tunes", it was re-released in July 2004 as the first single from the album "Twice the Speed of Life". The single reached a peak position of number 2 on the "Billboard" Hot Country Singles & Tracks (now Hot Country Songs) charts in April 2005. The song spent a total of 46 weeks on the country music charts; setting a new record for the longest chart run since the inception of Nielsen SoundScan digital chart tabulation in 1990. In addition, "Baby Girl" became the highest-peaking debut single for a country music group in 14 years. It was written by group members Jennifer Nettles, Kristian Bush, and Kristen Hall, along with Troy Bieser. Paragraph 2: Heaven

...

Yes


2024-02-13 22:32:01,258 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-13 22:32:01,260 - INFO - Paragraph 1: Red vs. Blue: Recreation Red vs. Blue: Recreation is the seventh season of the action comedy science fiction machinima web series Red vs. Blue created by Rooster Teeth Productions that premiered on June 9, 2009, and concluded on October 26, 2009. Set three days after the "" mini-series, the Red team are back plotting against the Blue team, which currently consists of only one person, Caboose. New episodes were released every Monday at 9:00 CST. Paragraph 2: John Prosky John Prosky is an American film, theatre, and television actor. His numerous TV credits include "NYPD Blue", "ER", "Heroes", "Criminal Minds", "True Blood", "JAG", "My So-Called Life", "", "The Practice", "The X-Files", "The West Wing", "Charmed", "24", "House", "Grey's Anatomy", "Veronica Mars", "Fringe," and the web series "Red Bird". His film credits include "Th

Haitian


2024-02-13 22:32:02,345 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-13 22:32:02,347 - INFO - ['#3', 'Jaguar', '65th', 'Lady Antebellum', 'Architect', '18 minutes', 'Judges', 'Winners', 'United States v. Paramount Pictures, Inc.', 'January', 'Wolf Creek', '1884', 'Yes', 'MacFarlane', 'FAI Junior Cup', 'Seasiders', '1970', 'Wollongong', 'Austrian', 'paranormal', 'Elizabeth Taylor', 'Crested Butte', 'California Polytechnic State University', 'May 22', 'Heard', 'ENnies', '4080', 'Wolfgang Becker', 'running back', 'Spring', "The Handmaid's Tale", 'Yeatman-Eiffel', 'Rockland', 'Premier League Darts', 'Suining', '2007', 'Marvel', 'Yes', 'Collins', 'Hall', 'Rock', '27', 'Judge Doom', 'Goalkeeper', "O'Brien", 'Haileybury', 'YZF-R6', 'American', 'Finding Nemo', 'Ephedrine', 'Suðreyjar', 'Kentucky', 'Panthera leo spelaea', 'Yes', 'Abigail', 'Hershey', 'Red Rock West', 'Adventist World', "Eto'o", 'Neveldine and Taylor', 'Chutzpah', 'Illinois', '

caddy
Percentage of True values: 39.0%


## Validator

### Baseline

In [10]:
fn_sfx = "" # filename suffix, used when saving validator file
sys_prompt = "Answer 'TRUE' if the answer is correct and 'FALSE' if the answer is incorrect."

### Search for optimal prompt

In [6]:
fn_sfx = "_prmpt" # filename suffix, used when saving validator file
sys_prompt = "Think step by step. First generate your own answer to the question and then compare this with the provided answer. Check, then double check your thinking. The last word of your response should be 'TRUE' if the answer is correct and 'FALSE' if the answer is incorrect, given the question."

### Chain of Thought

In [9]:
fn_sfx = "_cot"
sys_prompt = "Answer 'TRUE' if the answer is correct and 'FALSE' if the answer is incorrect, explaining your reasoning before answering. Think step by step."

### Run validator

In [10]:
# gen_data_fn = f"results/{st_fn}.csv" # change this to the filename of the generator results
gen_data_fn = 'results/2024_02_13_hotpotqa_n100_gpt-4-1106-preview.csv'

df = pd.read_csv(gen_data_fn)
indices = df.iloc[:, 0].tolist()
indices = indices[:-1] # just because we manually added an additional row on the CSV when calculating scores

val_preds = [] # list of predictions for validator, reset after each set of runs
val_evals = [] # whether prediction matches answer


# Evaluate
for _, row in df.iterrows():
    idx = row['Index']
    ans = row['Prediction']
    gold = row['Manual Eval']
    
    item = data[idx]
    # if item['level'] == 'hard':
    question, context = item['question'], item['context']
    question_with_context = ""
    for i in range(len(context)):
        context_str = ''.join(context[i][1])
        question_with_context += f"Paragraph {i+1}: {context[i][0]} {context_str} "
    question_with_context += f"Q: {question}"
    question_with_context += f"A: {ans}"
    logger.info(question_with_context)
    msg = [{"role": "system", "content": sys_prompt}, {"role": "user", "content": question_with_context}]
    val_preds.append(gpt.gen(msgs=msg, tokens=750).content)
    val_evals.append(check_bool(val_preds[-1], gold))
    print(val_preds[-1])

# Print the predictions and answers
logger.info(val_preds)
logger.info(df['Manual Eval'].tolist())
percent_correct = (sum(val_evals) / len(val_evals)) * 100

print(f"Percentage of True values: {percent_correct}%")

# add validator predictions to column
df['Validator Pred'] = val_preds
df['Validator Eval'] = val_evals

# save the dataframe to csv
df.to_csv(f'results/{st_fn}_with_val{fn_sfx}.csv', index=False);

2024-02-15 20:48:11,367 - INFO - Paragraph 1: Girls! Girls! Girls! Girls! Girls! Girls! is a 1962 Golden Globe-nominated American musical comedy film starring Elvis Presley as a penniless Hawaiian fisherman who loves his life on the sea and dreams of owning his own boat. "Return to Sender", which reached #2 on the "Billboard" pop singles chart, is featured in the movie. The movie opened at #6 on the "Variety" box office chart and finished the year at #31 on the year-end list of the top-grossing movies of 1962. The movie earned $2.6 million at the box office. Paragraph 2: Doctor Who: Series 4 (soundtrack) Doctor Who: Series 4 is a soundtrack album released on 17 November 2008, containing incidental music that was used throughout the fourth series of the BBC science fiction television programme "Doctor Who". The music was composed by the British musical director Murray Gold and was orchestrated by his collaborator Ben Foster who had previously worked with him on the films "Alien Autopsy"

TRUE

The last sentence of Paragraph 9 states: "The film was a moderate commercial success domestically, making no. 3 on the British box office chart." Therefore, the 2006 film "Alien Autopsy" promoted by Ant & Dec made it to the #3 spot on the British box office chart.


2024-02-15 20:48:16,400 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-15 20:48:16,401 - INFO - Paragraph 1: Inez Dickens Inez E. Dickens is the Assembly member for the 70th District of the New York State Assembly. She is a Democrat. The district includes portions of El Barrio, Hamilton Heights, Harlem, Morningside Heights, the Upper West Side and Washington Heights in Manhattan. She formerly served on the New York City Council from 2006 to 2016. Paragraph 2: Pete Grannis Alexander B. "Pete" Grannis (born 1942/1943) is a former Commissioner of the New York State Department of Environmental Conservation (DEC). Before his tenure as Commissioner, he was a member of the New York State Assembly and represented District 65 as a member of the Democratic Party for the neighborhoods of the Upper East Side of Manhattan and Roosevelt Island. His firing by Governor David Paterson in October 2010 was controversial to many especially environmentalist

TRUE

Jaguar Land Rover Automotive PLC, the British multinational automotive company, has its headquarters in Whitley, Coventry, United Kingdom, as stated in paragraph 4.


2024-02-15 20:48:24,756 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-15 20:48:24,758 - INFO - Paragraph 1: Baby Girl (Sugarland song) "Baby Girl" is the debut single co-written and recorded by American country music group Sugarland. Originally released in 2003 as part of "Premium Quality Tunes", it was re-released in July 2004 as the first single from the album "Twice the Speed of Life". The single reached a peak position of number 2 on the "Billboard" Hot Country Singles & Tracks (now Hot Country Songs) charts in April 2005. The song spent a total of 46 weeks on the country music charts; setting a new record for the longest chart run since the inception of Nielsen SoundScan digital chart tabulation in 1990. In addition, "Baby Girl" became the highest-peaking debut single for a country music group in 14 years. It was written by group members Jennifer Nettles, Kristian Bush, and Kristen Hall, along with Troy Bieser. Paragraph 2: Heaven

...

TRUE

Both Wilco Melissant and Todd Field are involved in the film industry. Wilco Melissant is mentioned as a Dutch director who directed various arthouse video productions and is the chairman of Witchhunt Productions. Todd Field is described as an actor and three-time Academy Award-nominated filmmaker who has directed and written several films.


2024-02-15 21:00:13,995 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-15 21:00:13,996 - INFO - Paragraph 1: Red vs. Blue: Recreation Red vs. Blue: Recreation is the seventh season of the action comedy science fiction machinima web series Red vs. Blue created by Rooster Teeth Productions that premiered on June 9, 2009, and concluded on October 26, 2009. Set three days after the "" mini-series, the Red team are back plotting against the Blue team, which currently consists of only one person, Caboose. New episodes were released every Monday at 9:00 CST. Paragraph 2: John Prosky John Prosky is an American film, theatre, and television actor. His numerous TV credits include "NYPD Blue", "ER", "Heroes", "Criminal Minds", "True Blood", "JAG", "My So-Called Life", "", "The Practice", "The X-Files", "The West Wing", "Charmed", "24", "House", "Grey's Anatomy", "Veronica Mars", "Fringe," and the web series "Red Bird". His film credits include "Th

TRUE

Suzanne Simone Baptiste Louverture was the wife of Toussaint Louverture, the leader of the Haitian Revolution. This is stated in Paragraph 1 where it mentions that she was the wife of Toussaint Louverture.


2024-02-15 21:00:28,206 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-15 21:00:28,208 - INFO - ['TRUE\n\nThe last sentence of Paragraph 9 states: "The film was a moderate commercial success domestically, making no. 3 on the British box office chart." Therefore, the 2006 film "Alien Autopsy" promoted by Ant & Dec made it to the #3 spot on the British box office chart.', 'TRUE\n\nJaguar Land Rover Automotive PLC, the British multinational automotive company, has its headquarters in Whitley, Coventry, United Kingdom, as stated in paragraph 4.', "TRUE\n\nPete Grannis represented District 65 as a member of the New York State Assembly, according to Paragraph 2, which includes the neighborhoods of the Upper East Side of Manhattan and Roosevelt Island. Paragraph 5 informs us that Roosevelt Island is part of Manhattan, thus confirming that Pete Grannis represented the district that includes Roosevelt Island. Therefore, the district that include

FALSE

The actor from the web series "Red Bird" who is also known for a character in the popular television show "Seinfeld" is Armin Shimerman. As mentioned in Paragraph 6, Armin Shimerman played Kramer's caddy Stan on "Seinfeld". However, the question seems to suggest that the character's name is simply "caddy," which is imprecise; the character has a name, which is Stan. The correct answer would be that Armin Shimerman is known for playing the character Stan, who was Kramer's caddy, on "Seinfeld".
Percentage of True values: 73.0%


In [17]:
df = pd.read_csv(f'results/{st_fn}_with_val{fn_sfx}.csv')

pred_evals = []
for _, row in df.iterrows():
    pred_evals.append(check_bool(row['Validator Pred'], gold))

df['Validator Eval'] = pred_evals

# save the dataframe to csv
df.to_csv(f'results/{st_fn}_with_val{fn_sfx}.csv', index=False);